# Manga DCGAN

This notebook is an expriment of using DC-GAN (Deep Convolutional Generative Adverserial Network) to generate comic/manga characters.

In [2]:
%matplotlib inline

import pickle as pkl

import numpy as np
import tensorflow as tf

## Network Input

In [4]:
def model_inputs(real_dim, z_dim):
    input_real = tf.placeholder(tf.float32, (None, *real_dim), name='input_real')
    inut_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
    return input_real, input_z

## Generator

In [3]:
def generator(z, output_dim, reuse=False, alpha=0.2, training=True):
    with tf.variable_scope('generator', reuse=reuse):
        
        # First fully connected layer
        layer1 = tf.layers.dense(z, 4*4*1024)
        layer1 = tf.reshape(layer1, (-1, 4, 4, 1024))
        layer1 = tf.layers.batch_normalization(layer1, training=training)
        layer1 = tf.maximum(alpha * layer1, layer1) # Leaky ReLU
        
        # First conv layer
        layer2 = tf.layers.conv2d_transpose(layer1, 512, 5, strides=2, padding='same')
        layer2 = tf.layers.batch_normalization(layer2, training=training)
        layer2 = tf.maximum(alpha * layer2, layer2) # Leaky ReLU
        
        # Second conv layer
        layer3 = tf.layers.conv2d_transpose(layer2, 256, 5, strides=2, padding='same')
        layer3 = tf.layers.batch_normalization(layer3, training=training)
        layer3 = tf.maximum(alpha * layer3, layer3) # Leaky ReLU
        
        # Third conv layer
        layer4 = tf.layers.conv2d_transpose(layer3, 128, 5, strides=2, padding='same')
        layer4 = tf.layers.batch_normalization(layer4, training=training)
        layer4 = tf.maximum(alpha * layer4, layer4) # Leaky ReLU
        
        # Fourth conv layer
        layer5 = tf.layers.conv2d_transpose(layer4, 64, 5, strides=2, padding='same')
        layer5 = tf.layers.batch_normalization(layer5, training=training)
        layer5 = tf.maximum(alpha * layer5, layer5) # Leaky ReLU
        
        # Output layer, 128x128x3
        logits = tf.layers.conv2d_transpose(layer5, output_dim, 5, strides=2, padding='same')
        
        out = tf.tanh(logits)
        
        return out

## Discriminator

In [15]:
def discriminator(x, reuse=False, alpha=0.2):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Input is 128x128x3
        layer1 = tf.layers.conv2d(x, 256, 5, strides=2, padding='same')
        layer1 = tf.layers.batch_normalization(layer1, training=True)
        layer1 = tf.maximum(alpha * layer1, layer1)

        # 64x64x256
        layer2 = tf.layers.conv2d(layer1, 512, 5, strides=2, padding='same')
        layer2 = tf.layers.batch_normalization(layer2, training=True)
        layer2 = tf.maximum(alpha * layer2, layer2)

        # 32x32x512
        layer3 = tf.layers.conv2d_transpose(layer2, 1024, 5, strides=2, padding='same')
        layer3 = tf.layers.batch_normalization(layer3, training=True)
        layer3 = tf.maximum(alpha * layer3, layer3)

        # 16x16x1024
        layer4 = tf.layers.conv2d_transpose(layer3, 2048, 5, strides=2, padding='same')
        layer4 = tf.layers.batch_normalization(layer4, training=True)
        layer4 = tf.maximum(alpha * layer4, layer4)

        # 8x8x2048
        flat = tf.reshape(layer4, (-1, 8*8*2048))
        logits = tf.layers.dense(flat, 1)
        out = tf.sigmoid(logits)

        return out, logits

## Model Loss

In [7]:
def model_loss(input_real, input_z, output_dim, alpha=0.2):
    """
    Get the loss for the discriminator and generator
    :param input_real: Images from the real dataset
    :param input_z: Z input
    :param out_channel_dim: The number of channels in the output image
    :return: A tuple of (discriminator loss, generator loss)
    """
    g_model = generator(input_z, output_dim, alpha=alpha)
    d_model_real, d_logits_real = discriminator(input_real, alpha=alpha)
    d_model_fake, d_logits_fake = discriminator(g_model, reuse=True, alpha=alpha)
    
    g_loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, label=tf.ones_like(d_model_fake)))
    
    d_loss_real = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logtis_real, label=tf.ones_like(d_model_real)))
    d_loss_fake = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, label=tf.zeros_like(d_model_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    
    return d_loss, g_loss

## Optimizers

In [9]:
def model_opt(d_loss, g_loss, learning_rate, beta1):
    """
    Get optimization operations
    :param d_loss: Discriminator loss Tensor
    :param g_loss: Generator loss Tensor
    :param learning_rate: Learning Rate Placeholder
    :param beta1: The exponential decay rate for the 1st moment in the optimizer
    :return: A tuple of (discriminator training operation, generator training operation)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    
    # Optimize, Using Adam optimizer
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        d_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
        g_train_opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(g_loss, var_list=g_vars)

    return d_train_opt, g_train_opt

## Building the model

In [8]:
class GAN:
    def __init__(self, real_size, z_size, learning_rate, alpha=0.2, beta1=0.5):
        tf.reset_default_graph()
        
        # Create input place holders
        self.input_real, self.input_z = model_input(real_size, z_size)
        
        # Get the model losses
        self.d_loss, self.g_loss = model_loss(self.input_real, self.input_z, real_size[2], alpha=alpha)
        
        # Get the optimized parameters
        self.d_opt, self_g_opt = model_opt(self.d_loss, self.g_loss, learning_rate, beta1=beta1)      

In [ ]:
def train(model, dataset, epochs, batch_size, print_every=10, show_every=100, figsize=(5,5)):
    saver = tf.train.Saver() # Saver used to save the checkpoints
    samples, losses = [], [] # Outputs
    steps = 0 # This variable is for showing the generator images
    
    with tf.Session() as sess:
        # Initialize the variables, this is a 
        # standard tensorflow operation
        sess.run(tf.global_variables_initializer())
        
        # Loop through epochs...
        for e in range(epochs):
            for x, y in dataset.batches(batch_size):
                step += 1
                
                # Sample random noise for G
                batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))
                
                # Run optimizers
                _ = sess.run(model.d_opt, feed_dict={model.input_real: x, model.input_z: batch_z})
                _ = sess.run(model.g_opt, feed_dict={model.input_z: batch_z, model.input_real: x})
                
                # Display options
                if steps % print_every == 0:
                
                
                if steps % show_every == 0:
                    
                
                
        saver.save(sess, './checkpoints/generator.ckpt')  
        
        
    return losses, samples

## Hyperparameters

## Execution